In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.3/609.3 kB 7.5 MB/s eta 0:00:00


In [82]:
import os
import shutil
import re
import json
import pandas as pd

from PIL import Image
from ultralytics import YOLO

In [4]:
# 필요한 폴더 생성 및 이동
!mkdir -p dataset/train/images
!mkdir -p dataset/train/labels
%cd dataset/train

# 이미지 다운로드 및 압축 해제
!wget -c http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip -d images
!rm val2017.zip

# 주석 다운로드 및 압축 해제
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip -d labels
!rm annotations_trainval2017.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: images/val2017/000000577584.jpg  
 extracting: images/val2017/000000346905.jpg  
 extracting: images/val2017/000000433980.jpg  
 extracting: images/val2017/000000228144.jpg  
 extracting: images/val2017/000000041872.jpg  
 extracting: images/val2017/000000117492.jpg  
 extracting: images/val2017/000000368900.jpg  
 extracting: images/val2017/000000376900.jpg  
 extracting: images/val2017/000000352491.jpg  
 extracting: images/val2017/000000330790.jpg  
 extracting: images/val2017/000000384850.jpg  
 extracting: images/val2017/000000032735.jpg  
 extracting: images/val2017/000000197004.jpg  
 extracting: images/val2017/000000526751.jpg  
 extracting: images/val2017/000000041488.jpg  
 extracting: images/val2017/000000153632.jpg  
 extracting: images/val2017/000000501523.jpg  
 extracting: images/val2017/000000405691.jpg  
 extracting: images/val2017/000000040757.jpg  
 extracting: images/val2017/000000219485.jpg  
 extracting: images/val2

In [5]:
# 이미지 파일 이동
val_images_dir = '/content/dataset/train/images/val2017'
images_dir = '/content/dataset/train/images'

for filename in os.listdir(val_images_dir):
    src_path = os.path.join(val_images_dir, filename)
    dest_path = os.path.join(images_dir, filename)
    shutil.move(src_path, dest_path)

# 주석 파일 이동
val_annotations_dir = '/content/dataset/train/labels/annotations'
labels_dir = '/content/dataset/train/labels'

for filename in os.listdir(val_annotations_dir):
    src_path = os.path.join(val_annotations_dir, filename)
    dest_path = os.path.join(labels_dir, filename)
    shutil.move(src_path, dest_path)

# 불필요한 폴더 삭제
os.rmdir(val_images_dir)
os.rmdir(val_annotations_dir)

In [6]:
# 이미지 파일 이름 변경 (앞의 0 제거)
for filename in os.listdir(images_dir):
    old_path = os.path.join(images_dir, filename)
    match = re.search(r'(\d+)', filename)

    if match:
        new_filename = str(int(match.group(1))) + '.jpg'
        new_path = os.path.join(images_dir, new_filename)
        shutil.move(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")
    else:
        print(f"Skipped: {filename} (No digits found)")

Renamed: 000000125572.jpg -> 125572.jpg
Renamed: 000000356169.jpg -> 356169.jpg
Renamed: 000000061658.jpg -> 61658.jpg
Renamed: 000000420472.jpg -> 420472.jpg
Renamed: 000000053624.jpg -> 53624.jpg
Renamed: 000000125211.jpg -> 125211.jpg
Renamed: 000000008021.jpg -> 8021.jpg
Renamed: 000000530099.jpg -> 530099.jpg
Renamed: 000000352582.jpg -> 352582.jpg
Renamed: 000000091406.jpg -> 91406.jpg
Renamed: 000000422670.jpg -> 422670.jpg
Renamed: 000000096960.jpg -> 96960.jpg
Renamed: 000000425361.jpg -> 425361.jpg
Renamed: 000000235784.jpg -> 235784.jpg
Renamed: 000000308394.jpg -> 308394.jpg
Renamed: 000000403122.jpg -> 403122.jpg
Renamed: 000000127494.jpg -> 127494.jpg
Renamed: 000000366884.jpg -> 366884.jpg
Renamed: 000000297595.jpg -> 297595.jpg
Renamed: 000000125806.jpg -> 125806.jpg
Renamed: 000000563281.jpg -> 563281.jpg
Renamed: 000000000285.jpg -> 285.jpg
Renamed: 000000201775.jpg -> 201775.jpg
Renamed: 000000577539.jpg -> 577539.jpg
Renamed: 000000572517.jpg -> 572517.jpg
Renamed: 

In [7]:
annotations_dir = '/content/dataset/train/labels'
json_file_path = os.path.join(annotations_dir, 'instances_val2017.json')

# JSON 파일 로드
with open(json_file_path, 'r') as f:
    coco_data = json.load(f)

# 이미지 정보 추출
image_info = coco_data['images']

# 주석 정보 추출
annotations = coco_data['annotations']

# 데이터프레임 생성을 위한 리스트 초기화
data = []

# 이미지 정보와 주석 정보를 결합하여 데이터프레임 생성
for ann in annotations:
    image_id = ann['image_id']
    image = [img for img in image_info if img['id'] == image_id][0]

    bbox = ann['bbox']
    category_id = ann['category_id']

    data.append({
        'image_id': image_id,
        'width': image['width'],
        'height': image['height'],
        'bbox': bbox,
        'category_id': category_id
    })

# 데이터프레임 생성
df = pd.DataFrame(data)

# 출력
df

,image_id,width,height,bbox,category_id
0,289343,529,640,"[473.07, 395.93, 38.65, 28.67]",18
1,61471,640,480,"[272.1, 200.23, 151.97, 279.77]",18
2,472375,612,612,"[124.71, 196.18, 372.85, 356.81]",18
3,520301,480,640,"[112.71, 154.82, 367.29, 479.35]",18
4,579321,640,498,"[200.61, 89.65, 400.22, 251.02]",18
...,...,...,...,...,...
36776,15517,640,480,"[197, 248, 264, 45]",6
36777,439994,428,640,"[0, 0, 427, 458]",1
36778,117719,640,425,"[6, 75, 474, 263]",44
36779,50149,500,376,"[10, 41, 403, 152]",52


In [8]:
# category_id가 17과 18인 값만 선택하여 새로운 DataFrame 생성
filtered_df = df[df['category_id'].isin([17, 18])]

# category_id 값을 변경하여 새로운 열 추가
filtered_df['category'] = filtered_df['category_id'].apply(lambda x: 0 if x == 17 else 1)

# 필요한 열만 선택하여 결과 출력
final_df = filtered_df[['image_id', 'width', 'height', 'bbox', 'category']]

# 인덱스 재설정
final_df = final_df.reset_index(drop=True)

final_df

<ipython-input-8-33225ed5e34b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['category'] = filtered_df['category_id'].apply(lambda x: 0 if x == 17 else 1)


,image_id,width,height,bbox,category
0,289343,529,640,"[473.07, 395.93, 38.65, 28.67]",1
1,61471,640,480,"[272.1, 200.23, 151.97, 279.77]",1
2,472375,612,612,"[124.71, 196.18, 372.85, 356.81]",1
3,520301,480,640,"[112.71, 154.82, 367.29, 479.35]",1
4,579321,640,498,"[200.61, 89.65, 400.22, 251.02]",1
...,...,...,...,...,...
415,57760,500,313,"[291.17, 211.0, 9.28, 4.96]",1
416,117908,500,320,"[346.84, 146.58, 153.16, 169.98]",0
417,14007,640,426,"[340.25, 153.38, 76.63, 145.04]",0
418,290843,428,640,"[22.02, 246.37, 220.22, 256.0]",0


In [9]:
output_dir = '/content/dataset/train/labels'  # YOLO 형식의 txt 파일을 저장할 디렉토리

# 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 이미지 별로 처리
for image_id, group in final_df.groupby('image_id'):
    txt_filename = os.path.join(output_dir, f"{image_id}.txt")

    with open(txt_filename, 'w') as txt_file:
        for _, row in group.iterrows():
            category = row['category']
            bbox = row['bbox']
            width, height = row['width'], row['height']

            # YOLO 형식으로 변환
            center_x = (bbox[0] + bbox[2] / 2) / width
            center_y = (bbox[1] + bbox[3] / 2) / height
            width_normalized = bbox[2] / width
            height_normalized = bbox[3] / height

            txt_line = f"{category} {center_x:.6f} {center_y:.6f} {width_normalized:.6f} {height_normalized:.6f}\n"
            txt_file.write(txt_line)

    print(f"Generated: {txt_filename}")

print("YOLO format txt files generated.")

Generated: /content/dataset/train/labels/1675.txt
Generated: /content/dataset/train/labels/4795.txt
Generated: /content/dataset/train/labels/7386.txt
Generated: /content/dataset/train/labels/10363.txt
Generated: /content/dataset/train/labels/14007.txt
Generated: /content/dataset/train/labels/14831.txt
Generated: /content/dataset/train/labels/15497.txt
Generated: /content/dataset/train/labels/17029.txt
Generated: /content/dataset/train/labels/18833.txt
Generated: /content/dataset/train/labels/22192.txt
Generated: /content/dataset/train/labels/22892.txt
Generated: /content/dataset/train/labels/23272.txt
Generated: /content/dataset/train/labels/25560.txt
Generated: /content/dataset/train/labels/29393.txt
Generated: /content/dataset/train/labels/30494.txt
Generated: /content/dataset/train/labels/39769.txt
Generated: /content/dataset/train/labels/46378.txt
Generated: /content/dataset/train/labels/47121.txt
Generated: /content/dataset/train/labels/49269.txt
Generated: /content/dataset/train/

In [ ]:
images_dir = '/content/dataset/train/images'  # images 폴더
labels_dir = '/content/dataset/train/labels'  # labels 폴더

# labels 폴더에 있는 파일 이름 추출
labels_filenames = set()
for filename in os.listdir(labels_dir):
    labels_filenames.add(filename.split('.')[0])  # 확장자 제거한 파일 이름

# images 폴더에서 labels 폴더와 파일 이름이 다른 파일 제거
for filename in os.listdir(images_dir):
    if filename.split('.')[0] not in labels_filenames:
        file_path = os.path.join(images_dir, filename)
        os.remove(file_path)
        print(f"Deleted: {filename}")

print("Files in 'images' directory have been deleted if their names are not in 'labels' directory.")

In [ ]:
labels_dir = '/content/dataset/val/labels'  # .json 파일이 있는 디렉토리

# .json 파일 제거
for filename in os.listdir(labels_dir):
    if filename.endswith('.txt'):
        file_path = os.path.join(labels_dir, filename)
        os.remove(file_path)
        print(f"Deleted: {filename}")

print("JSON files in 'labels' directory have been deleted.")

In [68]:
# images 폴더 내 파일 개수
images_dir = '/content/dataset/val/images'
num_images_files = len(os.listdir(images_dir))

# labels 폴더 내 파일 개수
labels_dir = '/content/dataset/val/labels'
num_labels_files = len(os.listdir(labels_dir))

print(f"Number of files in 'images' directory: {num_images_files}")
print(f"Number of files in 'labels' directory: {num_labels_files}")

Number of files in 'images' directory: 349
Number of files in 'labels' directory: 349


In [59]:
source_images_dir = '/content/dataset/train/images'  # 복사할 images 폴더
source_labels_dir = '/content/dataset/train/labels'  # 복사할 labels 폴더
destination_images_dir = '/content/dataset/val/images'  # 붙여넣을 images 폴더
destination_labels_dir = '/content/dataset/val/labels'  # 붙여넣을 labels 폴더

# val 폴더 내에 images와 labels 폴더 생성
os.makedirs(destination_images_dir, exist_ok=True)
os.makedirs(destination_labels_dir, exist_ok=True)

# images 폴더 내 파일 복사
for filename in os.listdir(source_images_dir):
    source_path = os.path.join(source_images_dir, filename)
    destination_path = os.path.join(destination_images_dir, filename)
    shutil.copy(source_path, destination_path)

# labels 폴더 내 파일 복사
for filename in os.listdir(source_labels_dir):
    source_path = os.path.join(source_labels_dir, filename)
    destination_path = os.path.join(destination_labels_dir, filename)
    shutil.copy(source_path, destination_path)

print("train 폴더의 images와 labels 폴더를 val 폴더로 복사하여 붙여넣었습니다.")

train 폴더의 images와 labels 폴더를 val 폴더로 복사하여 붙여넣었습니다.


In [78]:
config_txt = '''
path: /content/dataset  # dataset root dir
train: /content/dataset/train/images # (relative to 'path')
val: /content/dataset/val/images # (relative to 'path')
test: /content/dataset/test/images # test images (optional)

# Classes
nc: 2
names:
  0: cat
  1: dog
'''
with open("/content/dataset/cat_dog.yaml", 'w') as f:
    f.write(config_txt)

In [73]:
model = YOLO('yolov8n.pt')

In [ ]:
results = model.train(data='/content/dataset/cat_dog.yaml', epochs=100, imgsz=640, device='cuda')

In [89]:
preds = model('/content/cd1.jpg')


image 1/1 /content/cd1.jpg: 480x640 1 dog, 215.3ms
Speed: 8.4ms preprocess, 215.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)


In [86]:
for p in preds:
    im_array = p.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image